In [5]:
import pandas as pd
from readFiles import fullRead
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from tabpfn import TabPFNClassifier

In [6]:
def predictSex(pathToTable):
    df = fullRead(pathToTable)
    enc = OrdinalEncoder()
    enc.fit(df[["Sweetener", "Time"]])
    df[["Sweetener", "Time"]] = enc.transform(df[["Sweetener", "Time"]])
    X, y = df.drop(["numVol", "Sex"], axis=1), df["Sex"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)

    classifier.fit(X_train, y_train)
    y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)

    print('Accuracy in sex clasiffication', pathToTable, accuracy_score(y_test, y_eval))

def predictSweet(pathToTable):
    df = fullRead(pathToTable, anthro=True)
    df = df[df["Time"] == "Final"]
    enc = OrdinalEncoder()
    enc.fit(df[["Sex"]])
    df[["Sex"]] = enc.transform(df[["Sex"]])
    X, y = df.drop(["numVol", "Sweetener", "Time"], axis=1), df["Sweetener"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)

    classifier.fit(X_train, y_train)
    y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)

    print('Accuracy in sweetener clasiffication', pathToTable, accuracy_score(y_test, y_eval))    

In [7]:
predictSex("data/urineFlav_ord.csv")
predictSweet("data/urineFlav_ord.csv")
predictSex("data/urineAnt_ord.csv")
predictSweet("data/urineAnt_ord.csv")
predictSex("data/plasmFlav_ord.csv")
predictSweet("data/plasmFlav_ord.csv")
predictSex("data/plasmAnt_ord.csv")
predictSweet("data/plasmAnt_ord.csv")

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sex clasiffication data/urineFlav_ord.csv 0.6024096385542169
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sweetener clasiffication data/urineFlav_ord.csv 0.5476190476190477
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sex clasiffication data/urineAnt_ord.csv 0.6746987951807228
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sweetener clasiffication data/urineAnt_ord.csv 0.47619047619047616
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sex clasiffication data/plasmFlav_ord.csv 0.4878048780487805
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sweetener clasiffication data/plasmFlav_ord.csv 


  Cloning https://github.com/tusharsarkar3/XBNet.git to c:\users\cristina garcía\appdata\local\temp\pip-req-build-0y3c_1na
  Resolved https://github.com/tusharsarkar3/XBNet.git to commit 2b771d6beb9f15ae48e2a365f46d6f996811086f
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for XBNet: filename=XBNet-1.4.6-py3-none-any.whl size=13280 sha256=a46789c12094b72168a164e378dadc75a5bae2c5fa1a6c597b0a6e82a6cb45c8
  Stored in directory: C:\Users\Cristina García\AppData\Local\Temp\pip-ephem-wheel-cache-h219i07p\wheels\e1\fc\fd\454f766aabc107e8f137228c629835b432808c0ce974eff641
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=61ec43ac0a17595c11abc48fe8dcdfdfd23e8299ebefe8a935086712615eee93
  Stored in directory: c:\users\cristina garcía\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built XBNet sklearn
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalli

  Running command git clone -q https://github.com/tusharsarkar3/XBNet.git 'C:\Users\Cristina García\AppData\Local\Temp\pip-req-build-0y3c_1na'
ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'c:\\programdata\\anaconda3\\lib\\site-packages\\numpy-1.22.4.dist-info\\entry_points.txt'
Consider using the `--user` option or check the permissions.



sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x21956cbac40>
c:\ProgramData\Anaconda3\lib\threading.py:914: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  del self._target, self._args, self._kwargs
c:\ProgramData\Anaconda3\lib\threading.py:914: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  del self._target, self._args, self._kwargs
c:\ProgramData\Anaconda3\lib\threading.py:914: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  del self._target, self._args, self._kwargs
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x21956cbab80>
